In [52]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

In [53]:
postings_1 = pd.read_csv('input/Postings_01052019_15052019.csv', sep=';')
postings_2 = pd.read_csv('input/Postings_16052019_31052019.csv', sep=';')
votes_1 = pd.read_csv('input/Votes_01052019_15052019.csv', sep=';')
votes_2 = pd.read_csv('input/Votes_16052019_31052019.csv', sep=';')
follow_ignore = pd.read_csv(
    "input/Following_Ignoring_Relationships_01052019_31052019.csv", sep=";")

# Merge the two datasets
postings = pd.concat([postings_1, postings_2])
votes = pd.concat([votes_1, votes_2])

In [54]:
postings

,ID_Posting,ID_Posting_Parent,ID_CommunityIdentity,PostingHeadline,PostingComment,PostingCreatedAt,ID_Article,ArticlePublishingDate,ArticleTitle,ArticleChannel,ArticleRessortName,UserCommunityName,UserGender,UserCreatedAt
0,1041073586,1.041073e+09,671476,Das hat gestern bereits der Voggenhuber angefü...,schieder hatte dem inhaltlich nichts entgegenz...,2019-05-01 18:21:15.127,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Ravenspower,NaN,2018-04-14 13:42:28.470
1,1041073839,1.041073e+09,566938,NaN,...und meinen Bezirk bekommst du als Erbe mit.,2019-05-01 18:28:22.040,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,AlphaRomeo,m,2015-08-28 17:07:41.110
2,1041073872,1.041069e+09,669286,NaN,"Nein, bei der ÖVP/FPÖ genauso passiert. Ich wo...",2019-05-01 18:29:05.533,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Hpolditsch,NaN,2018-03-06 20:03:42.737
3,1041080734,1.041080e+09,671476,Sie haben doch nichts gefordert??,sie haben nur die regierung kritisiert. das di...,2019-05-01 22:37:56.010,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Ravenspower,NaN,2018-04-14 13:42:28.470
4,1041080828,NaN,671476,Heute wäre der perfekte Tag für die SPÖ gewese...,"ihr noch nicht erfülltes versprechen, den silb...",2019-05-01 22:42:06.310,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Ravenspower,NaN,2018-04-14 13:42:28.470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395929,1042380731,1.042380e+09,553628,NaN,Vermutlich gar keines...mir ist jedenfalls kei...,2019-06-04 08:54:54.177,2000103620997,2019-05-24 09:01:27.00,Off-Topic-Forum #123,User,Off-Topic,404 not found,NaN,2014-12-06 15:15:06.280
395930,1042381030,NaN,553628,NaN,*winkt dankbar zur Gödelnummer* Du bist echt d...,2019-06-04 09:04:32.037,2000103620997,2019-05-24 09:01:27.00,Off-Topic-Forum #123,User,Off-Topic,404 not found,NaN,2014-12-06 15:15:06.280
395931,1042381528,1.042381e+09,553628,NaN,"Die sind noch in Arbeit, aber der Surface läuf...",2019-06-04 09:22:54.473,2000103620997,2019-05-24 09:01:27.00,Off-Topic-Forum #123,User,Off-Topic,404 not found,NaN,2014-12-06 15:15:06.280
395932,1042381793,1.042380e+09,553628,NaN,"Ich versteh das überhaupt nicht, warum so viel...",2019-06-04 09:31:45.077,2000103620997,2019-05-24 09:01:27.00,Off-Topic-Forum #123,User,Off-Topic,404 not found,NaN,2014-12-06 15:15:06.280


In [55]:
postings['ArticleRessortName'].nunique()

689

In [56]:
postings['ID_CommunityIdentity'].nunique()

23925

In [76]:
postings_sub = postings[['ID_CommunityIdentity', 'ArticleRessortName']].drop_duplicates()
postings_sub

,ID_CommunityIdentity,ArticleRessortName
0,671476,Parteien
1,566938,Parteien
2,669286,Parteien
6,523527,Parteien
7,74674,Parteien
...,...,...
395753,105234,Inland
395755,224383,Wirtschaftspolitik
395758,547903,Wirtschaftspolitik
395760,694412,Jobwelten


In [109]:
user_categories = {}
for user in postings_sub['ID_CommunityIdentity'].unique():
    
    user_categories[user] = set(postings_sub.query("ID_CommunityIdentity == @user")['ArticleRessortName'].values)

In [ ]:
def jaqquard_similarity(a, b):
    return len(a & b) / len(a | b)

In [110]:
arr = []
for user_a in user_categories.keys():
    for user_b in user_categories.keys():
        if user_a != user_b:
            arr.append([user_a, user_b, jaqquard_similarity(user_categories[user_a], user_categories[user_b])])

arr

In [77]:
#Ana's sad attempt to make smth

# import warnings
# warnings.filterwarnings("ignore")

# groups = postings_sub.groupby("ID_CommunityIdentity")

# all_groups = groups.groups.keys()
# groups_list = list(all_groups)
# list_len = len(groups_list)

# df = pd.DataFrame(columns=('UserA', 'UserB', 'topics_num'))

# for i in range (list_len):
#     for j in range(i+1, list_len):
#         id_a = groups_list[i]
#         id_b = groups_list[j]
#         group_a = groups.get_group(id_a)
#         group_b = groups.get_group(id_b)
#         topics_num = len(np.intersect1d(group_a.ArticleRessortName, group_b.ArticleRessortName))

#         print(str(id_a)+ ", "+ str(id_b)+ ": "+ str(topics_num))

#         df.append({'UserA': id_a, 'UserB': id_b, 'topics_num': topics_num}, ignore_index=True)

# df


30, 38: 0
30, 50: 0
30, 66: 0
30, 77: 1
30, 148: 0
30, 157: 0
30, 233: 1
30, 241: 1
30, 243: 0
30, 304: 1
30, 344: 1
30, 356: 1
30, 360: 0
30, 364: 1
30, 428: 0
30, 472: 2
30, 508: 0
30, 615: 0
30, 626: 0
30, 629: 0
30, 657: 1
30, 797: 0
30, 807: 0
30, 825: 0
30, 932: 1
30, 1004: 0
30, 1011: 1
30, 1218: 0
30, 1224: 0
30, 1227: 0
30, 1228: 0
30, 1238: 0
30, 1264: 1
30, 1277: 3
30, 1354: 1
30, 1381: 1
30, 1402: 1
30, 1437: 1
30, 1493: 1
30, 1569: 0
30, 1685: 0
30, 1706: 2
30, 1778: 1
30, 1782: 2
30, 1880: 0
30, 1940: 1
30, 2051: 0
30, 2069: 0
30, 2071: 1
30, 2093: 1
30, 2130: 0
30, 2148: 1
30, 2177: 1
30, 2183: 1
30, 2191: 1
30, 2273: 0
30, 2278: 0
30, 2305: 1
30, 2336: 0
30, 2337: 0
30, 2359: 0
30, 2373: 1
30, 2422: 1
30, 2423: 0
30, 2512: 0
30, 2582: 1
30, 2722: 0
30, 2792: 0
30, 2809: 2
30, 2820: 3
30, 2968: 2
30, 3072: 0
30, 3457: 1
30, 3466: 0
30, 3467: 1
30, 3470: 1
30, 3479: 1
30, 3481: 2
30, 3499: 0
30, 3706: 0
30, 3719: 0
30, 3728: 0
30, 3758: 0
30, 3961: 1
30, 3976: 0
30, 4006:

KeyboardInterrupt: 